In [1]:
!pip install pandas vaderSentiment textblob transformers torch nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import pipeline

# Load dataset
input_file = "english_mahakumbh_data.csv"
df = pd.read_csv(input_file)

# Drop rows with missing text values
df.dropna(subset=["text"], inplace=True)

# Convert 'text' to string to avoid errors
df["text"] = df["text"].astype(str).str.strip()

# Initialize sentiment analysis models
vader_analyzer = SentimentIntensityAnalyzer()
bert_analyzer = pipeline("sentiment-analysis")

# Function for VADER sentiment analysis
def get_vader_score(text):
    return vader_analyzer.polarity_scores(text)["compound"]

# Function for TextBlob sentiment analysis
def get_textblob_score(text):
    return TextBlob(text).sentiment.polarity

# Function for BERT sentiment analysis
def get_bert_score(text):
    text = text[:512]  # Limit text length to 512 tokens for BERT
    if not text:  # Handle empty strings
        return 0

    result = bert_analyzer(text)
    label = result[0]["label"].lower()
    score = result[0]["score"]

    return score if label == "positive" else -score if label == "negative" else 0

# Apply sentiment analysis functions
df["VADER Score"] = df["text"].apply(get_vader_score)
df["TextBlob Score"] = df["text"].apply(get_textblob_score)
df["BERT Score"] = df["text"].apply(get_bert_score)

# Compute average sentiment score
df["Average Sentiment"] = df[["VADER Score", "TextBlob Score", "BERT Score"]].mean(axis=1)

# Function to assign sentiment label
def assign_label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment labeling
df["Sentiment Label"] = df["Average Sentiment"].apply(assign_label)

# Keep only necessary columns
df = df[["pseudo_id", "text", "VADER Score", "TextBlob Score", "BERT Score", "Average Sentiment", "Sentiment Label"]]

# Save the results to a new CSV file
output_file = "sentiment_analysis_mahakumbh.csv"
df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed and saved in '{output_file}'.")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Sentiment analysis completed and saved in 'sentiment_analysis_mahakumbh.csv'.


In [4]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from docx import Document
from docx.shared import Inches
import nltk
import os

# Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

# --- Setup ---
input_csv = "sentiment_analysis_mahakumbh.csv"
output_dir = "sentiment_report_visuals"
report_file = "Mahakumbh_Sentiment_Analysis_Report.docx"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
df = pd.read_csv(input_csv)

# Count sentiment classes
sentiment_counts = df["Sentiment Label"].value_counts()
total = len(df)
positive_count = sentiment_counts.get("Positive", 0)
neutral_count = sentiment_counts.get("Neutral", 0)
negative_count = sentiment_counts.get("Negative", 0)
dominant_sentiment = sentiment_counts.idxmax()

# --- Generate Bar Chart ---
sns.set(style="whitegrid")
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="Sentiment Label", order=["Positive", "Neutral", "Negative"], palette="pastel")
plt.title("Sentiment Distribution (Bar Chart)")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.tight_layout()
bar_chart_path = f"{output_dir}/sentiment_bar_chart.png"
plt.savefig(bar_chart_path)
plt.close()

# --- Generate Pie Chart ---
colors = ["#8fd694", "#f7d794", "#ff6b6b"]
plt.figure(figsize=(6, 6))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct="%1.1f%%", startangle=140, colors=colors)
plt.axis("equal")
plt.title("Sentiment Distribution (Pie Chart)")
pie_chart_path = f"{output_dir}/sentiment_pie_chart.png"
plt.savefig(pie_chart_path)
plt.close()

# --- Generate Word Clouds ---
stop_words = set(stopwords.words("english"))

def generate_wordcloud(sentiment_label, color_map):
    text_data = " ".join(df[df["Sentiment Label"] == sentiment_label]["text"].dropna().astype(str))
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color="white",
        stopwords=STOPWORDS.union(stop_words),
        colormap=color_map
    ).generate(text_data)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud - {sentiment_label}")
    cloud_path = f"{output_dir}/wordcloud_{sentiment_label.lower()}.png"
    plt.savefig(cloud_path)
    plt.close()
    return cloud_path

positive_wc = generate_wordcloud("Positive", "Greens")
neutral_wc = generate_wordcloud("Neutral", "Blues")
negative_wc = generate_wordcloud("Negative", "Reds")

# --- Create DOCX Report ---
doc = Document()
doc.add_heading("Sentiment Analysis Report", 0)

# Overview
doc.add_heading("Overview", level=1)
doc.add_paragraph(
    f"This report summarizes sentiment analysis results from the file '{input_csv}', "
    f"which includes {total} text entries. Sentiment labels were generated using VADER, "
    f"TextBlob, and BERT scores. The final label is based on the average sentiment score."
)

# Distribution Charts
doc.add_heading("Sentiment Distribution", level=1)
doc.add_picture(bar_chart_path, width=Inches(5.5))
doc.add_paragraph("Bar chart showing the count of Positive, Neutral, and Negative sentiments.")

doc.add_picture(pie_chart_path, width=Inches(5.5))
doc.add_paragraph("Pie chart showing sentiment proportions.")

# Word Clouds
doc.add_heading("Word Clouds", level=1)

doc.add_paragraph("Word Cloud - Positive")
doc.add_picture(positive_wc, width=Inches(5.5))

doc.add_paragraph("Word Cloud - Neutral")
doc.add_picture(neutral_wc, width=Inches(5.5))

doc.add_paragraph("Word Cloud - Negative")
doc.add_picture(negative_wc, width=Inches(5.5))

# Observations
doc.add_heading("Key Observations", level=1)
doc.add_paragraph(f"• Most of the texts were classified as **{dominant_sentiment}**.")
doc.add_paragraph(f"• Positive: {positive_count}")
doc.add_paragraph(f"• Neutral: {neutral_count}")
doc.add_paragraph(f"• Negative: {negative_count}")
doc.add_paragraph("• The combination of three sentiment models ensures a more reliable classification.")

# Save Report
doc.save(report_file)
print(f"✅ DOCX report saved as '{report_file}'")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-5-8515661f0f8f>:34: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x="Sentiment Label", order=["Positive", "Neutral", "Negative"], palette="pastel")


✅ DOCX report saved as 'Mahakumbh_Sentiment_Analysis_Report.docx'
